In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#Expects timeseries.py in same folder as nb. 
from fastai.vision import *
from pathlib import Path
import pdb
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from timeseries import TimeSeriesItem, TimeSeriesList, UCRArchive
from scipy.signal import resample
from IPython.display import clear_output
import fastai.utils.mem

In [3]:
#init UCR archive helper class
ucr = UCRArchive()

In [4]:
#All input is resampled down to a length of 96
class Resample(PreProcessor):
    def process_one(self,item):
        return np.concatenate([item[[0]],resample(item[1:],96)]) if len(item) > 97 else item

In [5]:
#Helper functions, modified to 1d from fastai
def create_head_1d(nf:int, nc:int, lin_ftrs:Optional[Collection[int]]=None, ps:Floats=0.5, bn_final:bool=False):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and about `nc` classes."
    lin_ftrs = [nf, 512, nc] if lin_ftrs is None else [nf] + lin_ftrs + [nc]
    ps = listify(ps)
    if len(ps)==1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    for ni,no,p,actn in zip(lin_ftrs[:-1],lin_ftrs[1:],ps,actns):
        layers += bn_drop_lin(ni,no,True,p,actn)
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    return nn.Sequential(*layers)

def conv1d(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias=False, init:LayerFunc=nn.init.kaiming_normal_) -> nn.Conv1d:
    "Create and initialize `nn.Conv1d` layer. `padding` defaults to `ks//2`."
    if padding is None: padding = ks//2
    return init_default(nn.Conv1d(ni, nf, kernel_size=ks, stride=stride, padding=padding, bias=bias), init)

In [6]:
#This is the hero network, which serves as a backbone to all the other models
class HeroConvnet(nn.Module):
    def __init__(self, num_layers=8, start_nf=8):
        super().__init__()
        
        layers = [nn.Sequential(conv1d(1,start_nf,3,1),nn.ReLU())] #First layer is stride 1, creates initial set of filters
        nf = start_nf
        for _ in range(num_layers): #Then num_layers stride 2 convs, doubling the number of filters each layer
            layers.append(nn.Sequential(conv1d(nf,nf*2,3,2),nn.ReLU()))
            nf *= 2
        
        self.nf = nf
        self.layers = nn.ModuleList(layers)
        self.avg = nn.AdaptiveAvgPool1d(1)
            
            
    def forward(self, x):
        actvns = [x]
        
        for l in self.layers:
            actvns.append(l(actvns[-1]))

        return self.avg(actvns[-1]), actvns[1:]

In [7]:
class BasicClassifier(nn.Module):
    def __init__(self,numClasses):
        super().__init__()
        self.conv = HeroConvnet()
        self.out = create_head_1d(self.conv.nf,numClasses,ps=0.0)
              
    def forward(self,ts):
        ts = self.conv(ts.unsqueeze(1))[0].squeeze(-1)
        return self.out(ts)

In [8]:
#Basic variational autoencoder with hero net serving as encoder and a few linear layers as decoder
class TSAutoencoder(nn.Module):
    def __init__(self,seqLen,latentDim=12):
        super().__init__()
        self.conv = HeroConvnet()
        self.mean = torch.nn.Linear(self.conv.nf,latentDim)
        self.logvar = torch.nn.Linear(self.conv.nf,latentDim)
        self.out = create_head_1d(latentDim,seqLen,lin_ftrs=[256,512],ps=0.0)

    def forward(self,ts):
        seqLen = ts.shape[1]
        ts, _ = self.conv(ts.unsqueeze(1))
        ts = ts.squeeze(-1)

        mean, logvar = self.mean(ts), self.logvar(ts)
          
        ls = mean
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            ls = eps.mul(std).add_(mean)        
        return self.out(ls), mean, logvar

In [9]:
#The sidekick network mirrors the structure of the hero, but concats the output of each layer of the hero to the input of each 
#layer of the sidekick
class SidekickConvnet(nn.Module):
    def __init__(self, num_classes, num_layers=8, start_nf=8, start_nf_hero=8):
        super().__init__()
        
        self.hero = HeroConvnet(num_layers,start_nf_hero)
        
        layers = [nn.Sequential(conv1d(1,start_nf,3,1),nn.ReLU())] 
        nf = start_nf
        nf_hero = start_nf_hero
        for _ in range(num_layers):
            layers.append(nn.Sequential(conv1d(nf+nf_hero,nf*2,3,2),nn.ReLU()))
            nf *= 2
            nf_hero *= 2
        
        self.layers = nn.ModuleList(layers)
        self.avg = nn.AdaptiveAvgPool1d(1)
        self.out = create_head_1d(nf + nf_hero,num_classes,ps=0.0)
    
    def forward(self,ts):
        ts = ts.unsqueeze(1)
        pt, actvns = self.hero(ts)
        
        x = self.layers[0](ts)
        for l,a in zip(self.layers[1:],actvns):
            x = l(torch.cat([x,a],dim=1))
            
        x = torch.cat([self.avg(x),pt],dim=1).squeeze(-1)
        return self.out(x)

In [10]:
class VAELoss(torch.nn.Module):
    def __init__(self,beta=1.0):
        super().__init__()
        self.beta = beta
    def forward(self,p,target):
        pred,mean,logvar = p
        self.mse = torch.nn.functional.mse_loss(pred,target,reduction="sum")
        self.kld = self.beta * -0.5 * torch.sum(1+logvar-mean.pow(2)-logvar.exp())
        return self.mse + self.kld

In [11]:
@fastai.utils.mem.gpu_mem_restore
def EvaluateDataset(dataset_name):
    out = [dataset_name]
    
    try:
        src = TimeSeriesList.from_csv_list(ucr.get_csv_files(dataset_name),labelCol=0)#,processor=Resample())
        valIdxs = np.random.choice(len(src.items),int(len(src.items)*0.3),replace=False)
        data = src.split_by_idx(valIdxs)
        data = data.label_from_col()
        idxs = np.random.choice(len(data.x),size=len(data.x)//10,replace=False)
        bs = min(64,len(data.x)//50)
        data.x.items = data.train.x.items[idxs]
        data.y.items = data.train.y.items[idxs]
        data = data.databunch(bs=bs,num_workers=0)

        src = TimeSeriesList.from_csv_list(ucr.get_csv_files(dataset_name),labelCol=0)#,processor=Resample())
        dataAE = src.split_by_idx(valIdxs)
        dataAE = dataAE.label_from_self()
        dataAE = dataAE.databunch(bs=bs,num_workers=0)

        learnBase = Learner(data,BasicClassifier(data.train_ds.c),loss_func=F.cross_entropy,metrics=[accuracy])
        learnBase.fit_one_cycle(20,1e-3,wd=0.2)
        out.append(max([m[0].item() for m in learnBase.recorder.metrics]))

        learnAE = Learner(dataAE,TSAutoencoder(len(data.train_ds[0][0].data),latentDim=12),loss_func=VAELoss())
        learnAE.fit_one_cycle(20,1e-2)
        out.append(learnAE.validate(dataAE.train_dl)[0])
        
        learnSidekick = Learner(data,SidekickConvnet(data.train_ds.c), loss_func=F.cross_entropy,metrics=[accuracy],
                            callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learnSidekick.split([learnSidekick.model.hero,learnSidekick.model.layers[0],learnSidekick.model.out])
        learnSidekick.model.hero.load_state_dict(learnAE.model.conv.state_dict())
        learnSidekick.freeze_to(1)
        learnSidekick.fit_one_cycle(20,1e-3,wd=0.2)
        learnSidekick.fit_one_cycle(20,1e-4,wd=0.2)
        out.append(max([m[0].item() for m in learnSidekick.recorder.metrics]))
        
        learnAE.fit_one_cycle(200,1e-3)
        out.append(learnAE.validate(dataAE.train_dl)[0])


        learnDT = Learner(data,BasicClassifier(data.train_ds.c),loss_func=F.cross_entropy,metrics=[accuracy],
                     callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learnDT.split([*learnDT.model.conv.layers,learnDT.model.conv.avg,learnDT.model.out])
        learnDT.model.conv.load_state_dict(learnAE.model.conv.state_dict())
        learnDT.freeze_to(-1)
        learnDT.fit_one_cycle(20,1e-2)
        out.append(max([m[0].item() for m in learnDT.recorder.metrics]))
        learnDT.unfreeze()
        learnDT.fit_one_cycle(20,1e-3)
        out.append(max([m[0].item() for m in learnDT.recorder.metrics]))
        
        learnSidekickBase = Learner(data,SidekickConvnet(data.train_ds.c), loss_func=F.cross_entropy,metrics=[accuracy],
                    callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learnSidekickBase.fit_one_cycle(20,1e-3,wd=0.2)
        learnSidekickBase.fit_one_cycle(20,1e-4,wd=0.2)
        out.append(max([m[0].item() for m in learnSidekickBase.recorder.metrics]))

        learnSidekick = Learner(data,SidekickConvnet(data.train_ds.c), loss_func=F.cross_entropy,metrics=[accuracy],
                            callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learnSidekick.split([learnSidekick.model.hero,learnSidekick.model.layers[0],learnSidekick.model.out])
        learnSidekick.model.hero.load_state_dict(learnAE.model.conv.state_dict())
        learnSidekick.freeze_to(1)
        learnSidekick.fit_one_cycle(20,1e-3,wd=0.2)
        learnSidekick.fit_one_cycle(20,1e-4,wd=0.2)
        out.append(max([m[0].item() for m in learnSidekick.recorder.metrics]))
    except:
        pass
    return out

In [12]:
results = []
skip = ["ACSF1","AllGestureWiimoteX","AllGestureWiimoteY","AllGestureWiimoteZ"]
for ds in ucr.list_datasets():
    if ds in skip: continue
    print(f"Evaluating {ds}")
    #for _ in range(4): results.append(EvaluateDataset(ds))
    results.append(EvaluateDataset(ds))
    clear_output()

In [13]:
results

[['Adiac',
  0.3717948794364929,
  58.85842,
  0.4017094075679779,
  26.876024,
  0.14102564752101898,
  0.1794871836900711,
  0.4188034236431122,
  0.38461539149284363],
 ['ArrowHead',
  0.7777777910232544,
  67.830734,
  0.6666666865348816,
  66.81082,
  0.4920634925365448,
  0.5079365372657776,
  0.7777777910232544,
  0.7460317611694336],
 ['Beef'],
 ['BeetleFly'],
 ['BirdChicken'],
 ['BME',
  0.9444444179534912,
  95.975204,
  0.7592592835426331,
  101.59677,
  0.5185185074806213,
  0.7407407164573669,
  0.7777777910232544,
  0.7037037014961243],
 ['Car'],
 ['CBF',
  0.9749103784561157,
  462.54697,
  0.9749103784561157,
  424.61813,
  0.9928315281867981,
  0.9964157938957214,
  0.939068078994751,
  0.9856630563735962],
 ['Chinatown',
  0.9266055226325989,
  nan,
  0.7247706651687622,
  nan,
  0.7247706651687622,
  0.7247706651687622,
  0.963302731513977,
  0.7247706651687622],
 ['ChlorineConcentration',
  0.6006191968917847,
  389.72275,
  0.7221361994743347,
  445.20932,
  0.5735

In [15]:
df = pd.DataFrame(results,columns=["Name","Baseline","AE20","Sidekick20","AE200","Direct","Fine Tune","SidekickBase","Sidekick"])

In [16]:
df

,Name,Baseline,AE20,Sidekick20,AE200,Direct,Fine Tune,SidekickBase,Sidekick
0,Adiac,0.371795,58.858421,0.401709,26.876024,0.141026,0.179487,0.418803,0.384615
1,ArrowHead,0.777778,67.830734,0.666667,66.810822,0.492063,0.507937,0.777778,0.746032
2,Beef,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BeetleFly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BirdChicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BME,0.944444,95.975204,0.759259,101.596771,0.518519,0.740741,0.777778,0.703704
6,Car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CBF,0.974910,462.546967,0.974910,424.618134,0.992832,0.996416,0.939068,0.985663
8,Chinatown,0.926606,NaN,0.724771,NaN,0.724771,0.724771,0.963303,0.724771
9,ChlorineConcentration,0.600619,389.722748,0.722136,445.209320,0.573529,0.582043,0.710526,0.688855


In [ ]:
df = df.groupby(by="Name").mean().reset_index()

In [17]:
df["Gain"] = df["Sidekick"] - df[["Baseline","SidekickBase"]].max(axis=1)

In [18]:
df["Sidekick Gain"] = df["Sidekick"] - df[["Direct","Fine Tune"]].max(axis=1)

In [ ]:
df.to_csv("results.csv")

In [ ]:
df.to_clipboard()

In [21]:
df.sort_values("Gain",ascending=False)

,Name,Baseline,AE20,Sidekick20,AE200,Direct,Fine Tune,SidekickBase,Sidekick,Gain,Sidekick Gain
107,SyntheticControl,0.783333,275.660461,0.894444,256.649139,0.788889,0.894444,0.755556,0.861111,0.077778,-0.033333
32,FaceAll,0.765926,1310.776611,0.834074,956.164246,0.746667,0.764444,0.789630,0.848889,0.059259,0.084444
78,PhalangesOutlinesCorrect,0.672522,89.742630,0.772898,8876.871094,0.715182,0.720201,0.772898,0.811794,0.038896,0.091593
71,MixedShapesRegularTrain,0.841505,2106.697998,0.860889,1811.907593,0.851767,0.867731,0.838084,0.880274,0.038769,0.012543
85,Plane,0.730159,91.518723,0.746032,91.032608,0.253968,0.301587,0.682540,0.761905,0.031746,0.460317
77,OSULeaf,0.310606,1075.557861,0.363636,697.996399,0.378788,0.393939,0.303030,0.340909,0.030303,-0.053030
72,MixedShapesSmallTrain,0.846764,1964.121704,0.873184,1231.103027,0.889036,0.887715,0.828269,0.871863,0.025099,-0.017173
101,SonyAIBORobotSurface1,0.967742,155.038452,0.989247,107.796333,0.989247,0.983871,0.951613,0.989247,0.021505,0.000000
56,InlineSkate,0.246154,1364.470581,0.241026,735.094849,0.276923,0.276923,0.230769,0.266667,0.020513,-0.010256
36,Fish,0.704762,67.874763,0.685714,73.091316,0.447619,0.495238,0.685714,0.723810,0.019048,0.228571
